## Imports

In [1]:
import numpy as np

from openseize.io.bases import Reader, Header, Writer
from openseize.io import edf, annotations
from openseize import demos
from openseize import producer

## Introduction

<font size=3>Openseize currently provides tools for reading and writing European Data Format (EDF) binary files. The details of this file specification can be found here: https://www.edfplus.info/specs/edf.html

<font size=3>This demo will describe how to open, read, and produce data from an EDF file using the <font color='darkcyan'><b>EDF Reader class</b></font> and write data to an EDF file using the <font color='darkcyan'><b>EDF Writer class</b></font>. Additionally, this demo will cover how to read Comma Separated (CSV) and Tab separated value (TSV) annotation text files and use the resulting annotations to mask produced EEG numpy arrays.   

## Reading EDF Files

<font size=3>Our goal is to use a Reader object to read from an EDF file. Let's first take a look at the help file available for the Reader class, to see what we'll need in order to create one.

In [2]:
help(edf.Reader)

Help on class Reader in module openseize.io.edf:

class Reader(openseize.io.bases.Reader)
 |  Reader(path)
 |  
 |  A reader of European Data Format (EDF/EDF+) files.
 |  
 |  The EDF specification has a header section followed by data records
 |  Each data record contains all signals stored sequentially. EDF+
 |  files include an annotation signal within each data record. To
 |  distinguish these signals we refer to data containing signals as
 |  channels and annotation signals as annotation. Currently, this reader
 |  does not support the reading of annotation signals.
 |  
 |  For details on the EDF/+ file specification please see:
 |  
 |  https://www.edfplus.info/specs/index.html
 |  
 |  Attributes:
 |      header: A dictionary representation of an EDF Header.
 |      shape: A tuple of channels, samples contained in this EDF
 |  
 |  Method resolution order:
 |      Reader
 |      openseize.io.bases.Reader
 |      abc.ABC
 |      openseize.core.mixins.ViewInstance
 |      builtin

<font size=3>The only parameter that is required to instantiate the Reader is a path to an EDF file. So we should pick a file to read off of. For these demos, we have stored demo data to a remote Zenodo repository. The demos module we imported has access to the files in this repo; we can see what's available by calling the <font color='firebrick'><i><b>available</b></i></font> method.

In [3]:
demos.paths.available

---Available demo data files & location---
------------------------------------------
annotations_001.txt            '/home/matt/python...nnotations_001.txt'
recording_001.edf              '/home/matt/python.../recording_001.edf'
5872_Left_group A.txt          '/home/matt/python...2_Left_group A.txt'
5872_Left_group A-D.edf        '/home/matt/python...Left_group A-D.edf'
write_test.edf                 '/home/matt/python...ata/write_test.edf'
CW0259_SWDs.npy                '/home/matt/python...ta/CW0259_SWDs.npy'
subset_001.edf                 '/home/matt/python...ata/subset_001.edf'


<font size=3> If the file is currently on your system, you'll see a local location after that file's name. If not, you'll see a link to the Zenodo repo. Regardless of its location, we can get access to a file by calling the <font color='firebrick'><i><b>locate</b></i></font> method. If the file hasn't already been found on your local machine, it will be downloaded to the demos data folder. This may take a few minutes, but will occur only once.

In [4]:
# Get access to the file's path locally, downloading if needed
filepath = demos.paths.locate('recording_001.edf')

In [5]:
# We can see the file's location on our local machine now that it has downloaded.
demos.paths.available

---Available demo data files & location---
------------------------------------------
annotations_001.txt            '/home/matt/python...nnotations_001.txt'
recording_001.edf              '/home/matt/python.../recording_001.edf'
5872_Left_group A.txt          '/home/matt/python...2_Left_group A.txt'
5872_Left_group A-D.edf        '/home/matt/python...Left_group A-D.edf'
write_test.edf                 '/home/matt/python...ata/write_test.edf'
CW0259_SWDs.npy                '/home/matt/python...ta/CW0259_SWDs.npy'
subset_001.edf                 '/home/matt/python...ata/subset_001.edf'


<font size=3>Now that we have our demo file path, we can pass it in to create our Reader object.

In [6]:
reader = edf.Reader(filepath)

### Properties and Attributes

<font size=3>To view the attributes and properties of this reader we can print the reader instance.

In [7]:
# Print out the reader object to see its attributes
print(reader)

Reader Object
---Attributes & Properties---
{'path': PosixPath('/home/matt/python/nri/openseize/src/openseize/demos/data/recording_001.edf'),
 'header': {'version': '0',
            'patient': 'PIN-42 M 11-MAR-1952 Animal',
            'recording': 'Startdate 15-AUG-2020 X X X',
            'start_date': '15.08.20',
            'start_time': '09.59.15',
            'header_bytes': 1536,
            'reserved_0': 'EDF+C',
            'num_records': 3775,
            'record_duration': 1.0,
            'num_signals': 5,
            'names': ['EEG EEG_1_SA-B', 'EEG EEG_2_SA-B', 'EEG EEG_3_SA-B',
                      'EEG EEG_4_SA-B', 'EDF Annotations'],
            'transducers': ['8401 HS:15279', '8401 HS:15279', '8401 HS:15279',
                            '8401 HS:15279', ''],
            'physical_dim': ['uV', 'uV', 'uV', 'uV', ''],
            'physical_min': [-8144.31, -8144.31, -8144.31, -8144.31, -1.0],
            'physical_max': [8144.319, 8144.319, 8144.319, 8144.319, 1.0],
  

<font size=3> The reader contains three attributes; a <font color=firebrick><b>path</b></font> to the open file, a dictionary containing the EDF's <font color=firebrick><b>header information</b></font>, and the <font color=firebrick><b>shape of the data</b></font>, represented as a 2-D numpy array, with channels along 0th axis and samples along the 1st axis.

<font size=3>The header dictionary contains all information stored to the header section of the EDF file. Details on the exact meaning of each of these fields can be found here: https://www.edfplus.info/specs/edf.html. To ease access to the header data, the header is a dict instance that has been extended to include '.' dot notation attribute access.

In [8]:
# Fetch the names of the channels using '.' dot notation
print(reader.header.names)

['EEG EEG_1_SA-B', 'EEG EEG_2_SA-B', 'EEG EEG_3_SA-B', 'EEG EEG_4_SA-B', 'EDF Annotations']


<font size=3>With the open reader instance, we can call the read method to read EDF data. To understand the parameters of this method lets ask for help.

In [9]:
help(reader.read)

Help on method read in module openseize.io.edf:

read(start, stop=None, padvalue=nan) method of openseize.io.edf.Reader instance
    Reads samples from this EDF for the specified channels.
    
    Args:
        start: int
            The start sample index to read.
        stop: int
            The stop sample index to read (exclusive). If None, samples
            will be read until the end of file. Default is None.
        padvalue: float
            Value to pad to channels that run out of samples to return.
            Only applicable if sample rates of channels differ. Default
            padvalue is NaN.
    
    Returns: 
        A float64 array of shape len(chs) x (stop-start) samples.



<font size=3>The Readers read method reads from a <font color=firebrick>start sample</font> to a <font color=firebrick>stop sample</font> within the file. If the stop sample is not given the reader will read to the end of the file.

In [10]:
# read samples 0 to 5 for all 4 channels
reader.read(0, 5)

array([[-19.87908032,   7.95793213,  19.88808032,  18.89390131,
         18.89390131],
       [-86.4890744 ,  51.70180884,  63.63195703,  88.48643243,
         63.63195703],
       [-85.49489539,  44.74255573,  29.82987048,  79.53882129,
         52.69598785],
       [ 62.63777802,  95.44568555,  77.55046326,  36.7891236 ,
        109.36419177]])

<font size=3>In addition to reading specific samples, the read method supports reading only a selection of <font color=firebrick>channels</font>.

In [11]:
#set the reader to read only channels 0 and 2
reader.channels = [0, 2]

# read samples 0 to 5 for channels 0 and 2
print(reader.read(0, 5))

#set the reader to read all channels again
reader.channels = reader.header.channels
print('Next read will read channels = ', reader.channels)

[[-19.87908032   7.95793213  19.88808032  18.89390131  18.89390131]
 [-85.49489539  44.74255573  29.82987048  79.53882129  52.69598785]]
Next read will read channels =  [0, 1, 2, 3]


<font size=3>The EDF file specification allows for signals that may be sampled at different sample rates to be stored to the same file. In this case, a signal will have fewer samples than other signals in the file. In order to return non-ragged numpy arrays, the Reader will append the value of the <font color=firebrick>padvalue</font> parameter to shorter signals so that all signals have the same length. This padvalue defaults to np.NaN but may take on any value useful for your analysis. 

### File Resources and Context Management

<font size=3>We have seen how to create a Reader instance and use it's read method to extract data from an EDF file. However, the file is still open and using resources that you need to recover. To do this you can call the Reader instance's <font color='firebrick'><b>close</b></font> method.

In [12]:
reader.close()

<font size=3>To address this potential resource leak, openseize supports opening files using context managment. What does this mean?
In python you open a text file using a piece of code that looks like this

```py
    with open('somefile.text', 'r') as infile:
        process file
```
    
<font size=3>When opened this way the file is automatically closed at the end of the "with" context. EDF Readers support opening EDF files in a context managed protocol too. <b>Here's how to open the file using the context manager protocol.</b>

In [13]:
# Open Reader as Context Manager and read data from within context
with edf.Reader(filepath) as cmreader:
    data = cmreader.read(0,)
    print(data[:5])

# Attempt to read from Reader after context has closed
try:
    cmreader.read(0,)
except ValueError as err:
    print("\nValueError:", err)

[[-1.98790803e+01  7.95793213e+00  1.98880803e+01 ...  4.50000000e-03
   4.50000000e-03  4.50000000e-03]
 [-8.64890744e+01  5.17018088e+01  6.36319570e+01 ...  4.50000000e-03
   4.50000000e-03  4.50000000e-03]
 [-8.54948954e+01  4.47425557e+01  2.98298705e+01 ...  4.50000000e-03
   4.50000000e-03  4.50000000e-03]
 [ 6.26377780e+01  9.54456855e+01  7.75504633e+01 ...  4.50000000e-03
   4.50000000e-03  4.50000000e-03]]

ValueError: seek of closed file


<font size=3>This method of opening files inside a specific context and performing operations on the data is the preferred way to work with files in Openseize since the resources are automatically recovered at the end of the context.

## Writing EDF Files

<font size=3>In addition to an EDF file Reader, Openseize provides an <font color=firebrick>EDF file writer</font>. One of the use cases for this Writer is to split an EDF with channels corresponding to multiple subjects into multiple EDFs containing channels for only one subject. For example, if your EDF contains 3 subjects with 4 channels, their is a total of 12 signals in the EDF. The Writer can then be used to write 3 files each containing 4 channels. Lets examine how to use this Writer. <b>We'll again start by asking for help.</b>

In [14]:
help(edf.Writer)

Help on class Writer in module openseize.io.edf:

class Writer(openseize.io.bases.Writer)
 |  Writer(path)
 |  
 |  A writer of European Data Format (EDF) files.
 |  
 |  This writer does not support writing annotations to an EDF file.
 |  
 |  Method resolution order:
 |      Writer
 |      openseize.io.bases.Writer
 |      abc.ABC
 |      openseize.core.mixins.ViewInstance
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, path)
 |      Initialize this Writer. See base class for futher details.
 |  
 |  write(self, header, data, channels, verbose=True)
 |      Write header & data for each channel to file object.
 |      
 |      Args:
 |          header: dict
 |              A mapping of EDF compliant fields and values. For Further
 |              details see Header class of this module.
 |          data: 2-D array or Reader instance
 |              A channels x samples array or Reader instance.
 |          channels: sequence
 |              A sequence of 

<font size=3>To construct a Writer instance you need to provide a file path where the writer will write the new EDF file to. The <font color=firebrick><b>write</b></font> method is what you will need to call in order to write data to the file path. <b>Lets examine this method by asking for the method's documentation.</b>

In [15]:
help(edf.Writer.write)

Help on function write in module openseize.io.edf:

write(self, header, data, channels, verbose=True)
    Write header & data for each channel to file object.
    
    Args:
        header: dict
            A mapping of EDF compliant fields and values. For Further
            details see Header class of this module.
        data: 2-D array or Reader instance
            A channels x samples array or Reader instance.
        channels: sequence
            A sequence of channel indices to write to this Writer's 
            open file instance.
        verbose: bool
            An option to print progress of write. Default (True) prints
            status update as each record is written.



<font size=3>To write an EDF compliant file, the write method will need an <font color=firebrick>EDF Header instance</font> with all required fields and values expected of the EDF file type. An enumeration of the required fields and values can be found by examining the header printed above or be reading the EDF file specification here: https://www.edfplus.info/specs/edf.html
    
<font size=3>In addition to an EDF compliant Header instance, the write method needs <font color=firebrick>data</font>. This data may be an in-memory or a reader instance from which data will be fetched.
    
<font size=3>Lastly, the write method can take a <font color=firebrick>list of channel indices</font>. These channel indices will be used to filter both the Header instance and the data. For example, if you provide a Header containing metadata for 4 signals and an array containing 4 signals, you can request to write out a subset of the signals, say channel indices [0, 2]. This allows for the splitting of a multichannel EDF into multiple EDFs. Importantly, both the new data written and the new Header will contain only data and metadata for the 2 channels written. <b>Lets demonstrate these ideas with an example.</b>

In [16]:
# Create path for new EDF file to save to
save_path = demos.paths.data_dir.joinpath('subset_001.edf')

# Create an EDF writer pointing to this path
writer = edf.Writer(save_path)
print(writer)

Writer Object
---Attributes & Properties---
{'path': PosixPath('/home/matt/python/nri/openseize/src/openseize/demos/data/subset_001.edf')}

Type help(Writer) for full documentation


<font size=3>The writer knows where it will write data to and the write method can now be called to perform the writing. We will write channels 0 and 2 from the 'recording_001.edf' we used earlier. Since this file has a header, we will reuse that header. The write method will select metadata from the header corresponding to channels 0 and 2. The method will also only write data records corresponding to channels 0 and 2. <b><i>Remember to open the reader as a context manager so the file resources are automatically recovered.</i></b>

In [17]:
#locate the path to the recording
fp = demos.paths.locate('recording_001.edf')

#open the reader as context manager
with edf.Reader(fp) as reader:
    
    #open the writer as a context manager
    with edf.Writer(save_path) as writer:
        
        #write channels 0 and 2 from the header and reader's data
        writer.write(reader.header, reader, channels=[0,2])
        

<font size=3>Notice here that we called both the Reader and Writer as context managers. Just like reader instances, writer instances maintain an open file to write to that is using your machines resources. By opening both the reader and writer as context managers, these file resources will be closed when the reading and writing is finished.</font>

<font size=3><b>Now let's reopen the 'subset_001.edf' file we just wrote and make sure the header and data looks correct.</b>

In [18]:
with edf.Reader(save_path) as reader:
    
    # lets print the readers Header-- it should only have metadata for channels 0 and 2
    print('---EDF SUBSET HEADER---')
    print(reader.header)
    
    #lets print the first 5 samples and check these against the full data
    print('---EDF SUBSET DATA---')
    print(reader.read(0,5))

---EDF SUBSET HEADER---
{'version': '0',
 'patient': 'PIN-42 M 11-MAR-1952 Animal',
 'recording': 'Startdate 15-AUG-2020 X X X',
 'start_date': '15.08.20',
 'start_time': '09.59.15',
 'header_bytes': 768,
 'reserved_0': 'EDF+C',
 'num_records': 3775,
 'record_duration': 1.0,
 'num_signals': 2,
 'names': ['EEG EEG_1_SA-B', 'EEG EEG_3_SA-B'],
 'transducers': ['8401 HS:15279', '8401 HS:15279'],
 'physical_dim': ['uV', 'uV'],
 'physical_min': [-8144.31, -8144.31],
 'physical_max': [8144.319, 8144.319],
 'digital_min': [-8192.0, -8192.0],
 'digital_max': [8192.0, 8192.0],
 'prefiltering': ['none', 'none'],
 'samples_per_record': [5000, 5000],
 'reserved_1': ['', '']}

{'Accessible Properties': ['annotated', 'annotation', 'channels', 'offsets',
                           'record_map', 'samples', 'slopes']}
---EDF SUBSET DATA---
[[-19.87908032   7.95793213  19.88808032  18.89390131  18.89390131]
 [-85.49489539  44.74255573  29.82987048  79.53882129  52.69598785]]


<font size=3>Both the header and the data appear to contain only the metadata and data for channels 0 and 2. Now lets check that is the case by examining all the data against the original 'recording_001.edf' demo file.</font>

In [19]:
#fp is the still the filepath to recording_001.edf
with edf.Reader(fp) as reader:
    
    #read all 4 channels from the file
    all_data = reader.read(0)

#save_path is where the subset_001.edf resides
with edf.Reader(save_path) as reader:
    
    #read the 2 channels from the subset file
    two_ch_data = reader.read(0)
    
print("Do the arrays match? -> ", np.allclose(all_data[[0,2], :], two_ch_data))

Do the arrays match? ->  True


## EDF Annotations

<font size=3>In addition to EDF file readers, Openseize provides <font color=firebrick>annotation file readers</font>. Typically, annotation files are comma-separated or tab-separated value text files that contain time-stamps and labels of important events that occurred during an EEG recording session. Here we will show how to open a Pinnacle format annotation TSV text file. <b>Let's start by looking at the documentation for this annotation reader.</b>

In [20]:
help(annotations.Pinnacle)

Help on class Pinnacle in module openseize.io.annotations:

class Pinnacle(openseize.io.bases.Annotations)
 |  Pinnacle(path, **kwargs)
 |  
 |  A Pinnacle Technologies© annotations file reader.
 |  
 |  Pinnacle files store annotation data to a plain text file. This
 |  reader reads each row of this file extracting and storing annotation
 |  data to a sequence Annotation objects one per annotation (row) in the
 |  file.
 |  
 |  Method resolution order:
 |      Pinnacle
 |      openseize.io.bases.Annotations
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  channel(self, row)
 |      Extracts the annotation channel for a row in this file.
 |  
 |  duration(self, row)
 |      Measures the duration of an annotation for a row in this file.
 |  
 |  label(self, row)
 |      Extracts the annotation label for a row in this file.
 |  
 |  open(self, path, start=0, delimiter='\t', **kwargs)
 |      Opens a file returning a file handle and row iterator.
 |      
 

<font size=3>To construct an annotation reader you will need to provide a <font color=firebrick>path to an annotation file</font>. This path is given to the open method (see above). Additionally, you may need to pass in a <font color=firebrick>start line of the file</font>. This describes what line the column data starts on. Lets fetch the demo file "annotations_001.txt" if it is not on your system already and display the file's contents.

In [21]:
#determine the local path using the locate method and download if necessary
annotations_path = demos.paths.locate('annotations_001.txt')

#lets take a look at the file
with open(annotations_path, 'r') as infile:
    for idx, row in enumerate(infile):
        print(idx, row)

0 Experiment ID	Experiment

1 Animal ID	Animal

2 Researcher	Test

3 Directory path	

4 

5 

6 Number	Start Time	End Time	Time From Start	Channel	Annotation

7 0	08/15/20 09:59:15.215	08/15/20 09:59:15.215	0.0000	ALL	Started Recording

8 1	08/15/20 10:00:00.000	08/15/20 10:00:00.000	44.7850	ALL	Qi_start

9 2	08/15/20 10:00:25.000	08/15/20 10:00:30.000	69.7850	ALL	grooming

10 3	08/15/20 10:00:45.000	08/15/20 10:00:50.000	89.7850	ALL	grooming

11 4	08/15/20 10:02:15.000	08/15/20 10:02:20.000	179.7850	ALL	grooming

12 5	08/15/20 10:04:36.000	08/15/20 10:04:41.000	320.7850	ALL	exploring

13 6	08/15/20 10:05:50.000	08/15/20 10:05:55.000	394.7850	ALL	exploring

14 7	08/15/20 10:08:50.000	08/15/20 10:08:55.000	574.7850	ALL	rest

15 8	08/15/20 10:10:14.000	08/15/20 10:10:19.000	658.7850	ALL	exploring

16 9	08/15/20 10:17:10.000	08/15/20 10:17:15.000	1074.7850	ALL	rest

17 10	08/15/20 10:35:49.000	08/15/20 10:35:54.000	2193.7850	ALL	rest

18 11	08/15/20 10:40:00.000	08/15/20 10:40:00.000	2444

<font size=3>With this path  we can now construct an Annotations reader instance. Just as with Readers and Writers <b><i>an instance can (and most of the time should) be constructed as a context manager.</i></b> Below we are going to construct the annotations reader starting from line 6 since that is the row containing the column headers of the file. Note this initialization argument is passed to the open method which can accept any argument that python's builtin CSV.DictReader can accept.

In [22]:
#open the annotations and read all the annotations in the file using the 'read' method
with annotations.Pinnacle(annotations_path, start=6) as reader:
    
    #call read to get the annotations as a sequence of Annotation instances (to be described in a moment)
    annotes = reader.read()
    
#print the sequence of annotation instances
for instance in annotes:
    print(instance)

Annotation(label='Started Recording', time=0.0, duration=0.0, channel='ALL')
Annotation(label='Qi_start', time=44.785, duration=0.0, channel='ALL')
Annotation(label='grooming', time=69.785, duration=5.0, channel='ALL')
Annotation(label='grooming', time=89.785, duration=5.0, channel='ALL')
Annotation(label='grooming', time=179.785, duration=5.0, channel='ALL')
Annotation(label='exploring', time=320.785, duration=5.0, channel='ALL')
Annotation(label='exploring', time=394.785, duration=5.0, channel='ALL')
Annotation(label='rest', time=574.785, duration=5.0, channel='ALL')
Annotation(label='exploring', time=658.785, duration=5.0, channel='ALL')
Annotation(label='rest', time=1074.785, duration=5.0, channel='ALL')
Annotation(label='rest', time=2193.785, duration=5.0, channel='ALL')
Annotation(label='Qi_stop', time=2444.785, duration=0.0, channel='ALL')
Annotation(label='Stopped Recording', time=3774.664, duration=0.0, channel='ALL')


<font size=3>You can see that we have fetched all of the annotations from the displayed file and stored each annotation to an <font color=firebrick>Annotation instance</font>. What is this instance? An Annotation object is a python dataclass. If you haven't seen this before, you can think of it as a simple container with '.' dot notation access to the container's contents. <b>Let's examime the third dataclass instance.</b>

In [23]:
#fetch the third annotation item and display it
item = annotes[3]
print(item)

#access the items time from recording start
print('This annotation occurred at {} s relative to the start time'.format(item.time))

Annotation(label='grooming', time=89.785, duration=5.0, channel='ALL')
This annotation occurred at 89.785 s relative to the start time


<font size=3>The key pieces of information are given to us in a single Annotation instance:
 - <font color=firebrick><b>label</b></font> - a piece of text describing the annotation
 - <font color=firebrick><b>time</b></font> - the exact point in time (in seconds) from the beginning of the recording that the annotation takes place
 - <font color=firebrick><b>duration</b></font> - the length (in seconds) of the annotation from its start time 
 - <font color=firebrick><b>channel</b></font> - a list of the channels in the EEG recording that the annotation is applied to

<font size=3>In the preceding example, we read all of the annotations from the Pinnacle formatted file but the Annotations 'read' method can accept a sequence of <font color=firebrick>labels</font> to selectively read only some of the annotations. <b>Let's show how this works on this demo annotation file.</b>

In [24]:
#read only the annotations with labels matching either rest or exploring
with annotations.Pinnacle(annotations_path, start=6) as reader:
    subset_annotes = reader.read(labels=['rest', 'exploring'])
    
for annote in subset_annotes:
    print(annote)

Annotation(label='exploring', time=320.785, duration=5.0, channel='ALL')
Annotation(label='exploring', time=394.785, duration=5.0, channel='ALL')
Annotation(label='rest', time=574.785, duration=5.0, channel='ALL')
Annotation(label='exploring', time=658.785, duration=5.0, channel='ALL')
Annotation(label='rest', time=1074.785, duration=5.0, channel='ALL')
Annotation(label='rest', time=2193.785, duration=5.0, channel='ALL')


## Producing from EDF Files with Annotations 

<font size=3>Two important components of an Annotation instance is the <font color=firebrick><b>time</b></font> and <font color=firebrick><b>duration</b></font> attributes. These attributes allow for selective filtering of EEG data returned from either a Reader or a producer. To do this, the annotation dataclass  instances are converted into a boolean mask that can pick out samples of data to keep or discard. <b>Here we will demonstrate how to construct a boolean mask from a list of annotation instances and use that mask to filter a producer's yielded numpy arrays.</b> Further details can be found in the producer demo. 

<font size=3>The annotations module provides a method for generating a mask automatically from a series of annotation objects, the <font color=firebrick><b>as_mask</b></font> method.

In [25]:
help(annotations.as_mask)

Help on function as_mask in module openseize.io.annotations:

as_mask(annotations, size, fs, include=True)
    Convert a sequence of annotation objects into a 1-D boolean array. 
    
    Args:
        annotations: list
            A sequence of annotation objects to convert to a mask.
        size: int
            The length of the boolean array to return.
        fs: int
            The sampling rate in Hz of the recorded EEG.
        include: bool
            A boolean to determine if annotations should be set to True or
            False in the returned array. Default is True, meaning all values
            are False in the returned array except for samples where the
            annotations are located.
    
    Returns:
        A 1-D boolean array of length size.



<font size=3>To construct a mask, <font color=firebrick><b>as_mask</b></font> needs a sequence of <font color=firebrick>annotation dataclass instances</font>, the <font color=firebrick>size</font> of the mask along the sample axis, the <font color=firebrick>sampling rate</font> to convert the annotation times to samples, and a <font color=firebrick>boolean "include" parameter</font> which determines if the annotations should be kept (True) or discarded (False) from the EEG data.
<font size=3>Here, as an example, <b>we create such a mask.</b></font>

In [26]:
#Read the annotations from the demo file
with annotations.Pinnacle(annotations_path, start=6) as reader:
    subset_annotes = reader.read(labels=['rest', 'exploring'])

In [27]:
#Build the mask; fp is the still the filepath to recording_001.edf; size and fs can be fetched from reader
with edf.Reader(fp) as reader:    
    size = reader.shape[-1]
    fs = reader.header.samples_per_record[0]
    
mask = annotations.as_mask(subset_annotes, size, fs, include=True)

In [28]:
#print the first 10 values of the mask
print(mask[:10])

#The first True values should occur at 320.785 secs * 5000 Hz since fs=5000 and the first annotation 
#(see above occurs at 320.785). Lets confirm this by print 10 samples around this sample
start = int(320.785 * 5000)
print(mask[start-5: start+5])

#lastly lets print out the total number of samples we will keep
expected = len(subset_annotes) * 5 * 5000 # each annote is 5 secs @ 5 kHz
actual = np.count_nonzero(mask)
print('Expected number of samples to keep is {} \nActual number kept is {}'.format(expected, actual))

[False False False False False False False False False False]
[False False False False False  True  True  True  True  True]
Expected number of samples to keep is 150000 
Actual number kept is 150000


In [29]:
# Build a producer with this mask and show that it has the expected shape
with edf.Reader(filepath) as reader:
    masked_producer = producer(reader.read(start=6), chunksize=500, axis=-1, mask=mask)   
    print("Producer Shape (w/ mask applied):",masked_producer.shape)

Producer Shape (w/ mask applied): (4, 150000)


<font size=3>As we can see, the producer maintains four channels, with 150000 records each, exactly as we anticipated from our mask.